In [34]:
# Regression Example With Boston Dataset: Standardized 
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#load data
path = 'C:/Users/mys12/Desktop/northeastern/summer2018/data_Robust_Fingerprinting-master/DISTRIBUTED_OPENSOURCE/FINGERPRINTING_DB'

test_rss = pd.read_csv(path + '/Training_rss_21Aug17.csv', header = 0)
test_coord = pd.read_csv(path + '/Training_coordinates_21Aug17.csv', header = 0)
test_rss = test_rss.replace(100, 0)

train_rss = pd.read_csv(path + '/Test_rss_21Aug17.csv', header = 0)
train_coord = pd.read_csv(path + '/Test_coordinates_21Aug17.csv', header = 0)
train_rss = train_rss.replace(100, 0)

In [59]:
# train 
train_r = train_rss.values
train_ch = train_coord.iloc[:,-1]
building_floors_str = train_ch.map(str)  #convert all the building floors to strings
print("train_labels: ", np.unique(building_floors_str, return_counts=True))

train_labels = np.asarray(building_floors_str)
#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_label = pd.get_dummies(train_labels)
train_labels = np.asarray(dummy_label)

normalizer = preprocessing.Normalizer().fit(np.asarray(train_r))
train_r_features = normalizer.transform(np.asarray(train_r))

# test
test_r = test_rss.values
test_ch = test_coord.ix[:,-1]
test_labels = np.asarray(test_ch.map(str))
print("test_labels: ", np.unique(test_labels, return_counts=True))

test_labels = np.asarray(pd.get_dummies(test_labels))

#Scale transforms data to center to the mean and component wise scale to unit variance
test_r_features = normalizer.transform(np.asarray(test_r))
print(test_r_features.shape)


print(train_r_features.shape)

train_labels:  (array(['0.0', '11.1', '14.8', '3.7', '7.4'], dtype=object), array([1264,  699,  109, 1108,  770], dtype=int64))
test_labels:  (array(['0.0', '11.1', '14.8', '3.7', '7.4'], dtype=object), array([226, 118,  17, 197, 138], dtype=int64))
(696, 992)
(3950, 992)


C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [60]:
# Create a random subsample from the dataset with replacement 
from random import seed 
from random import randrange

def subsample(x,y, ratio=0.5): 
    sample_x = list() 
    sample_y = list()
    n_sample = round(len(x) * ratio) 
    while len(sample_x) < n_sample: 
        index = randrange(len(x)) 
        sample_x.append(x[index]) 
        sample_y.append(y[index])
    sample_x = np.array(sample_x)
    sample_y = np.array(sample_y)
    return sample_x, sample_y

In [61]:
# train1_x, train1_y = subsample(train_r_features, train_labels)
# train2_x, train2_y = subsample(train_r_features, train_labels)
# train3_x, train3_y = subsample(train_r_features, train_labels)
# train4_x, train4_y = subsample(train_r_features, train_labels)
# train5_x, train5_y = subsample(train_r_features, train_labels)

In [62]:
nb_epochs = 100
batch_size = 64
input_size = 992
num_classes = 5

In [63]:
def classifier():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax', bias=True))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [64]:
def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.predict(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [65]:
def accuracy(l1,l2):
    a=0
    for i in l1, j in l2:
        if i == j:
            a=a+1
    return a / len(l1)    

In [66]:

def train_val(train_Xx, train_Yy):
    train_x, val_x, train_y, val_y = train_test_split(train_Xx, train_Yy, test_size=0.2)
    return train_x, val_x, train_y, val_y

In [153]:
# print accuarcy of every subsample and get's its prediction
l1 = list()
for i in range(5):
    train_x, train_y = subsample(train_r_features, train_labels)
    train_x2, val_x2, train_y2, val_y2 = train_val(train_x, train_y)
    model = classifier()
    model.fit(train_x2, train_y2, validation_data=(val_x2, val_y2), nb_epoch=nb_epochs, batch_size=batch_size)
    loss, acc = model.evaluate(test_r_features, test_labels)
    l1.append(model.predict_classes(test_r_features))
    print("for subsample %d, its accuarcy is %.4f" % (i,acc)) 
  

C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=992, activation="tanh", use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  """
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", use_bias=True)`
  import sys
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1580 samples, validate on 395 samples
Epoch 1/100
1580/1580 [==============================] - 2s - loss: 1.0618 - acc: 0.6209 - val_loss: 0.6637 - val_acc: 0.7443
Epoch 2/100
1580/1580 [==============================] - 0s - loss: 0.5717 - acc: 0.7715 - val_loss: 0.4468 - val_acc: 0.8329
Epoch 3/100
1580/1580 [==============================] - 0s - loss: 0.4028 - acc: 0.8475 - val_loss: 0.3347 - val_acc: 0.8937
Epoch 4/100
1580/1580 [==============================] - 0s - loss: 0.3078 - acc: 0.8899 - val_loss: 0.2813 - val_acc: 0.9038
Epoch 5/100
1580/1580 [==============================] - 0s - loss: 0.2576 - acc: 0.9051 - val_loss: 0.2681 - val_acc: 0.9215
Epoch 6/100
1580/1580 [==============================] - 0s - loss: 0.2165 - acc: 0.9247 - val_loss: 0.2619 - val_acc: 0.9114
Epoch 7/100
1580/1580 [==============================] - 0s - loss: 0.1921 - acc: 0.9342 - val_loss: 0.2317 - val_acc: 0.9190
Epoch 8/100
1580/1580 [==============================] - 0s - loss: 0.1

1580/1580 [==============================] - 0s - loss: 0.0293 - acc: 0.9892 - val_loss: 0.3765 - val_acc: 0.9266
Epoch 66/100
1580/1580 [==============================] - 0s - loss: 0.0206 - acc: 0.9905 - val_loss: 0.3754 - val_acc: 0.9266
Epoch 67/100
1580/1580 [==============================] - 0s - loss: 0.0154 - acc: 0.9956 - val_loss: 0.3712 - val_acc: 0.9266
Epoch 68/100
1580/1580 [==============================] - 0s - loss: 0.0190 - acc: 0.9930 - val_loss: 0.3813 - val_acc: 0.9266
Epoch 69/100
1580/1580 [==============================] - 0s - loss: 0.0184 - acc: 0.9962 - val_loss: 0.3797 - val_acc: 0.9241
Epoch 70/100
1580/1580 [==============================] - 0s - loss: 0.0195 - acc: 0.9949 - val_loss: 0.3930 - val_acc: 0.9266
Epoch 71/100
1580/1580 [==============================] - 0s - loss: 0.0156 - acc: 0.9943 - val_loss: 0.3681 - val_acc: 0.9266
Epoch 72/100
1580/1580 [==============================] - 0s - loss: 0.0298 - acc: 0.9905 - val_loss: 0.3678 - val_acc: 0.92

1580/1580 [==============================] - 0s - loss: 0.0622 - acc: 0.9753 - val_loss: 0.2799 - val_acc: 0.9316
Epoch 29/100
1580/1580 [==============================] - 0s - loss: 0.0499 - acc: 0.9829 - val_loss: 0.2843 - val_acc: 0.9266
Epoch 30/100
1580/1580 [==============================] - 0s - loss: 0.0517 - acc: 0.9816 - val_loss: 0.3045 - val_acc: 0.9241
Epoch 31/100
1580/1580 [==============================] - 0s - loss: 0.0522 - acc: 0.9816 - val_loss: 0.2922 - val_acc: 0.9291
Epoch 32/100
1580/1580 [==============================] - 0s - loss: 0.0484 - acc: 0.9867 - val_loss: 0.2869 - val_acc: 0.9316
Epoch 33/100
1580/1580 [==============================] - 0s - loss: 0.0429 - acc: 0.9886 - val_loss: 0.3098 - val_acc: 0.9266
Epoch 34/100
1580/1580 [==============================] - 0s - loss: 0.0470 - acc: 0.9835 - val_loss: 0.3361 - val_acc: 0.9215
Epoch 35/100
1580/1580 [==============================] - 0s - loss: 0.0459 - acc: 0.9880 - val_loss: 0.3002 - val_acc: 0.93

1580/1580 [==============================] - 0s - loss: 0.0294 - acc: 0.9905 - val_loss: 0.6170 - val_acc: 0.9038
Epoch 93/100
1580/1580 [==============================] - 0s - loss: 0.0203 - acc: 0.9937 - val_loss: 0.5398 - val_acc: 0.9114
Epoch 94/100
1580/1580 [==============================] - 0s - loss: 0.0154 - acc: 0.9949 - val_loss: 0.5631 - val_acc: 0.9089
Epoch 95/100
1580/1580 [==============================] - 0s - loss: 0.0184 - acc: 0.9943 - val_loss: 0.5768 - val_acc: 0.8937
Epoch 96/100
1580/1580 [==============================] - 0s - loss: 0.0171 - acc: 0.9937 - val_loss: 0.5050 - val_acc: 0.9139
Epoch 97/100
1580/1580 [==============================] - 0s - loss: 0.0160 - acc: 0.9956 - val_loss: 0.5381 - val_acc: 0.9139
Epoch 98/100
1580/1580 [==============================] - 0s - loss: 0.0157 - acc: 0.9943 - val_loss: 0.5779 - val_acc: 0.9089
Epoch 99/100
1580/1580 [==============================] - 0s - loss: 0.0185 - acc: 0.9937 - val_loss: 0.5515 - val_acc: 0.90

1580/1580 [==============================] - 0s - loss: 0.0330 - acc: 0.9918 - val_loss: 0.1266 - val_acc: 0.9696
Epoch 56/100
1580/1580 [==============================] - 0s - loss: 0.0326 - acc: 0.9892 - val_loss: 0.1274 - val_acc: 0.9671
Epoch 57/100
1580/1580 [==============================] - 0s - loss: 0.0315 - acc: 0.9918 - val_loss: 0.1363 - val_acc: 0.9671
Epoch 58/100
1580/1580 [==============================] - 0s - loss: 0.0256 - acc: 0.9930 - val_loss: 0.1393 - val_acc: 0.9646
Epoch 59/100
1580/1580 [==============================] - 0s - loss: 0.0363 - acc: 0.9899 - val_loss: 0.1366 - val_acc: 0.9646
Epoch 60/100
1580/1580 [==============================] - 0s - loss: 0.0228 - acc: 0.9937 - val_loss: 0.1503 - val_acc: 0.9519
Epoch 61/100
1580/1580 [==============================] - 0s - loss: 0.0293 - acc: 0.9924 - val_loss: 0.1491 - val_acc: 0.9570
Epoch 62/100
1580/1580 [==============================] - 0s - loss: 0.0285 - acc: 0.9930 - val_loss: 0.1552 - val_acc: 0.95

1580/1580 [==============================] - 0s - loss: 0.1226 - acc: 0.9633 - val_loss: 0.2887 - val_acc: 0.9114
Epoch 19/100
1580/1580 [==============================] - 0s - loss: 0.1106 - acc: 0.9652 - val_loss: 0.2795 - val_acc: 0.9114
Epoch 20/100
1580/1580 [==============================] - 0s - loss: 0.1058 - acc: 0.9715 - val_loss: 0.2955 - val_acc: 0.9089
Epoch 21/100
1580/1580 [==============================] - 0s - loss: 0.1038 - acc: 0.9741 - val_loss: 0.2957 - val_acc: 0.9165
Epoch 22/100
1580/1580 [==============================] - 0s - loss: 0.0959 - acc: 0.9703 - val_loss: 0.3056 - val_acc: 0.9038
Epoch 23/100
1580/1580 [==============================] - 0s - loss: 0.0983 - acc: 0.9728 - val_loss: 0.3202 - val_acc: 0.8962
Epoch 24/100
1580/1580 [==============================] - 0s - loss: 0.0851 - acc: 0.9741 - val_loss: 0.3279 - val_acc: 0.9089
Epoch 25/100
1580/1580 [==============================] - 0s - loss: 0.0800 - acc: 0.9804 - val_loss: 0.3290 - val_acc: 0.91

1580/1580 [==============================] - 0s - loss: 0.0247 - acc: 0.9962 - val_loss: 0.6573 - val_acc: 0.9038
Epoch 83/100
1580/1580 [==============================] - 0s - loss: 0.0280 - acc: 0.9918 - val_loss: 0.6408 - val_acc: 0.9063
Epoch 84/100
1580/1580 [==============================] - 0s - loss: 0.0246 - acc: 0.9924 - val_loss: 0.6361 - val_acc: 0.9114
Epoch 85/100
1580/1580 [==============================] - 0s - loss: 0.0169 - acc: 0.9943 - val_loss: 0.6183 - val_acc: 0.9063
Epoch 86/100
1580/1580 [==============================] - 0s - loss: 0.0276 - acc: 0.9930 - val_loss: 0.6500 - val_acc: 0.9089
Epoch 87/100
1580/1580 [==============================] - 0s - loss: 0.0238 - acc: 0.9924 - val_loss: 0.6580 - val_acc: 0.9165
Epoch 88/100
1580/1580 [==============================] - 0s - loss: 0.0247 - acc: 0.9918 - val_loss: 0.6479 - val_acc: 0.9089
Epoch 89/100
1580/1580 [==============================] - 0s - loss: 0.0185 - acc: 0.9949 - val_loss: 0.6415 - val_acc: 0.90

1580/1580 [==============================] - 0s - loss: 0.0357 - acc: 0.9880 - val_loss: 0.2897 - val_acc: 0.9494
Epoch 46/100
1580/1580 [==============================] - 0s - loss: 0.0329 - acc: 0.9886 - val_loss: 0.3181 - val_acc: 0.9468
Epoch 47/100
1580/1580 [==============================] - 0s - loss: 0.0390 - acc: 0.9861 - val_loss: 0.2942 - val_acc: 0.9392
Epoch 48/100
1580/1580 [==============================] - 0s - loss: 0.0467 - acc: 0.9861 - val_loss: 0.3174 - val_acc: 0.9392
Epoch 49/100
1580/1580 [==============================] - 0s - loss: 0.0431 - acc: 0.9842 - val_loss: 0.3120 - val_acc: 0.9392
Epoch 50/100
1580/1580 [==============================] - 0s - loss: 0.0350 - acc: 0.9892 - val_loss: 0.3096 - val_acc: 0.9392
Epoch 51/100
1580/1580 [==============================] - 0s - loss: 0.0341 - acc: 0.9880 - val_loss: 0.3205 - val_acc: 0.9519
Epoch 52/100
1580/1580 [==============================] - 0s - loss: 0.0338 - acc: 0.9905 - val_loss: 0.3196 - val_acc: 0.93

In [154]:
l1[1]

array([3, 3, 3, 3, 4, 0, 4, 3, 0, 4, 3, 3, 3, 0, 1, 0, 4, 0, 4, 2, 1, 1, 0,
       0, 4, 0, 0, 4, 0, 3, 0, 0, 1, 4, 0, 4, 1, 4, 4, 3, 3, 1, 0, 1, 3, 1,
       0, 0, 4, 4, 3, 0, 0, 0, 4, 3, 3, 4, 2, 3, 3, 0, 0, 0, 1, 3, 3, 1, 4,
       3, 0, 4, 4, 4, 0, 3, 3, 0, 1, 3, 3, 0, 4, 0, 1, 0, 0, 3, 0, 4, 3, 3,
       0, 4, 4, 0, 0, 3, 3, 4, 0, 3, 4, 4, 3, 3, 2, 0, 3, 1, 1, 4, 3, 1, 4,
       0, 0, 3, 3, 3, 4, 0, 1, 3, 0, 0, 3, 3, 0, 3, 0, 4, 4, 3, 0, 0, 0, 4,
       0, 0, 0, 4, 0, 0, 1, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 0, 4, 0,
       0, 0, 0, 1, 4, 1, 0, 1, 3, 3, 0, 0, 4, 4, 0, 4, 3, 0, 3, 0, 3, 4, 4,
       1, 3, 4, 0, 1, 0, 4, 3, 0, 4, 3, 4, 2, 4, 3, 4, 4, 1, 0, 3, 4, 4, 2,
       0, 4, 1, 3, 2, 0, 0, 3, 0, 1, 4, 0, 1, 0, 1, 0, 1, 1, 4, 0, 0, 1, 3,
       3, 3, 1, 0, 0, 0, 0, 3, 1, 0, 1, 1, 2, 4, 4, 3, 0, 2, 0, 0, 3, 3, 0,
       1, 3, 3, 4, 1, 3, 0, 4, 0, 3, 4, 0, 4, 3, 4, 3, 0, 0, 0, 4, 1, 0, 3,
       3, 1, 4, 4, 3, 0, 1, 3, 0, 0, 0, 1, 3, 1, 0, 4, 4, 3, 0, 4, 3, 3, 0,
       0, 1,

In [155]:
l2 =list(map(list, zip(*l1)))

In [170]:
l2[208]

[3, 4, 4, 4, 4]

In [156]:
l2[1]
mode(l2[1])

3

In [157]:
from statistics import mode, StatisticsError
vote = []

for i in range(len(l2)):
    try:
        a = mode(l2[i])
        vote.append(a)
    except StatisticsError:
        vote.append(0)


In [158]:
vote = np.asarray(vote)
dummy_vote_label = pd.get_dummies(vote)
vote_labels = np.asarray(dummy_vote_label)

In [159]:
len(vote_labels)

696

In [160]:
vote_labels

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ..., 
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [161]:
from __future__ import division 
a =0

for i in range(696):
    if np.all(vote_labels[i]) == np.all(test_labels[i]):
        a = a + 1

print(a)
accuarcy = a/696
print(accuarcy)    

696
1.0


In [120]:
test_labels[0]

array([0, 0, 0, 1, 0], dtype=uint8)

In [ ]:
# accuracy = sum(YPred == YTest)/numel(YTest)